In [12]:
import pandas as pd
import numpy as np
import os
import dotenv
from dotenv import load_dotenv

In [13]:
import sys
import os

# Go one level up to the project root
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import common.PortConnect as Port_Connect

In [ ]:
import common.PortConnect as Port_Connect


ModuleNotFoundError: No module named 'common'